# Exploitation Zone - Visualització

- exploitation zone de la visualització
- preparation pipeline per taula d'entrenament del model --> cada zipcode és un indiv
    - Shops: latitu, longitud
    - Income: mitjana income per zipcode

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .config("spark.jars", "./duckdb.jar") \
        .appName("ExploitationZone") \
        .getOrCreate()



24/04/25 00:08:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
############
## INCOME ##
############

income = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone.duckdb") \
    .option("dbtable", "cleaned_income") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .load()

income_selected = income.select("ZIPCODE", "Total_income_amount")

In [3]:
###########
## SHOPS ##
###########
shops = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone.duckdb") \
    .option("dbtable", "cleaned_shops") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .load()

shops_selected = shops.select("ZIPCODE", "latitude", "longitude")

In [4]:
from pyspark.sql.functions import col

# taula a partir de income
df_model = income_selected

# renombrem columnes
df_model = df_model.withColumnRenamed("Total_income_amount", "avg_income_per_zipcode")



df_model.show()

+-------+----------------------+
|zipcode|avg_income_per_zipcode|
+-------+----------------------+
|  35004|                258024|
|  35005|                129390|
|  35006|                 58585|
|  35007|                651350|
|  35010|                382106|
|  35014|                 67885|
|  35016|                333226|
|  35019|                 35392|
|  35020|                262475|
|  35022|                521539|
|  35023|                480458|
|  35031|                112152|
|  35033|                 67437|
|  35034|                 52030|
|  35035|                 31542|
|  35040|                359868|
|  35042|                 96503|
|  35043|                363943|
|  35044|                124406|
|  35045|                236772|
+-------+----------------------+
only showing top 20 rows



In [5]:
joined_data = df_model.join(shops_selected, (df_model["ZIPCODE"] == shops_selected["ZIPCODE"]), "inner")
selected_columns = [df_model[col] for col in df_model.columns] + [shops_selected[col] for col in shops_selected.columns if col not in ["ZIPCODE"]]
joined_data = joined_data.select(selected_columns)
joined_data.show()

+-------+----------------------+------------------+-------------------+
|zipcode|avg_income_per_zipcode|          latitude|          longitude|
+-------+----------------------+------------------+-------------------+
|  96815|                993218|21.270510000000005|-157.80038400000004|
|  96815|                993218|21.284324499999986|        -157.814114|
|  96815|                993218| 21.28381529999999|        -157.814795|
|  96815|                993218| 21.28075830052637| -157.8279499995846|
|  96815|                993218|21.280566799999995|       -157.8281395|
|  96815|                993218| 21.27874419999999|        -157.813419|
|  96815|                993218| 21.27798179999999|       -157.8137586|
|  96815|                993218|21.276678100695744| -157.8146362996257|
|  96815|                993218|        21.2758429|         -157.81457|
|  96815|                993218|  21.2753013000051| -157.8175495998725|
|  96815|                993218|21.276000599999985|       -157.8

In [6]:
joined_data.write \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:exploitation_zone.duckdb") \
    .option("dbtable", "mapa_visualitzacio") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .mode("overwrite") \
    .save()
        